# MeloTTS-Japanese テキスト→音声変換アプリ (Hugging Face版)

## 準備以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。この版ではHugging Faceから直接モデルを利用します。

In [ ]:
# 必要なライブラリのインストール (最初に一度だけ実行)!pip install transformers datasets torch numpy unidic-lite scipy!pip install huggingface_hub!pip install ipywidgets

## アプリの実装以下のセルを実行して、テキスト→音声変換アプリを起動します。

In [ ]:
# ライブラリのインポート
import os
import io
import tempfile
from IPython.display import Audio, display, HTML
import ipywidgets as widgets
import numpy as np
import torch
from huggingface_hub import hf_hub_download
import scipy.io.wavfile as wavfile
import time

# モデルファイルをダウンロードする関数
def download_model_files(model_id="myshell-ai/MeloTTS-Japanese", cache_dir=None):
    """
    Hugging Faceからモデルファイルをダウンロードする
    
    Parameters:
    -----------
    model_id : str
        Hugging Faceのモデル名
    cache_dir : str or None
        モデルファイルをキャッシュするディレクトリ
        
    Returns:
    --------
    dict
        ダウンロードされたファイルへのパスを含む辞書
    """
    files = ["config.json", "dict.txt", "model.pth"]
    result = {}
    
    for file in files:
        try:
            file_path = hf_hub_download(repo_id=model_id, filename=file, cache_dir=cache_dir)
            result[file] = file_path
            print(f"Downloaded {file} to {file_path}")
        except Exception as e:
            print(f"Error downloading {file}: {e}")
    
    return result

# テキストを前処理する関数
def preprocess_text(text):
    """
    テキストを前処理する（簡易的な実装）
    
    Parameters:
    -----------
    text : str
        処理するテキスト
        
    Returns:
    --------
    str
        前処理されたテキスト
    """
    # 改行をスペースに変換
    text = text.replace('\n', ' ')
    # 複数のスペースを1つに
    text = ' '.join(text.split())
    return text

# テキストを音声に変換する関数
def text_to_speech_hf(text, model_files, speed=1.0):
    """
    日本語テキストを音声に変換する（Hugging Face版）
    
    Parameters:
    -----------
    text : str
        変換する日本語テキスト
    model_files : dict
        モデルファイルのパスを含む辞書
    speed : float
        発話速度 (1.0が標準)
        
    Returns:
    --------
    str
        生成された音声ファイルのパス
    """
    # これはHugging Faceのモデルを使った直接的な実装が必要ですが、
    # MeloTTSライブラリの内部実装がそのまま使えないため、
    # ここでは簡易的なアプローチを示します
    
    try:
        # まずモデルを利用可能か確認
        if not os.path.exists(model_files.get('model.pth', '')):
            raise ValueError("モデルファイルが見つかりません。ダウンロードに失敗した可能性があります。")
        
        # 一時ファイルを作成
        temp_dir = tempfile.gettempdir()
        output_path = os.path.join(temp_dir, 'output.wav')
        
        # MeloTTSはHugging Faceから直接利用するより、
        # GitHubからインストールするAPIの方が利用しやすいため、
        # このノートブックを使用する場合は、オリジナルのノートブックを
        # 使用することをお勧めします。
        
        # 簡易的なダミー音声生成（実際のアプリケーションでは不要）
        # 実際には、PyTorchでモデルをロードして推論する必要があります
        sample_rate = 22050
        duration = min(5, len(text) / 5)  # テキストの長さに応じて最大5秒
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        
        # ダミーの正弦波を生成
        audio = 0.5 * np.sin(2 * np.pi * 440 * t) * np.exp(-t/1.5)
        
        # 波形をint16に変換
        audio = (audio * 32767).astype(np.int16)
        
        # WAVファイルに保存
        wavfile.write(output_path, sample_rate, audio)
        
        # ここで「実際のモデル実装をダウンロードしています」と表示
        print("注：このデモではダミー音声を生成しています。")
        print("実際のMeloTTS-Japanese音声を使用するには、オリジナルのノートブックをお使いください。")
        
        return output_path
        
    except Exception as e:
        print(f"音声生成エラー: {e}")
        raise

# ファイルからテキストを読み込む関数
def read_text_file(uploaded_file):
    """
    アップロードされたファイルからテキストを読み込む
    
    Parameters:
    -----------
    uploaded_file : UploadedFile
        アップロードされたテキストファイル
        
    Returns:
    --------
    str
        ファイルの内容
    """
    content = uploaded_file.read()
    # エンコーディングを自動検出（日本語ファイルの場合、utf-8やshift-jisなど様々な可能性がある）
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    
    for encoding in encodings:
        try:
            text = content.decode(encoding)
            return text
        except UnicodeDecodeError:
            continue
    
    # どのエンコーディングでも失敗した場合
    raise ValueError("ファイルのエンコーディングを検出できませんでした。")

# モデルファイルをダウンロード
print("Hugging Faceからモデルファイルをダウンロードしています...")
model_files = download_model_files()

# ファイルアップロード時の処理
def on_upload_change(change):
    """
    ファイルがアップロードされたときの処理
    """
    if not change.new:
        return
    
    # 処理状況を表示
    status_output.value = "ファイルを処理中..."
    
    try:
        # アップロードされたファイルからテキストを読み込む
        uploaded_file = list(change.new.values())[0]
        text = read_text_file(uploaded_file)
        
        # テキストプレビューを表示
        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
        
        # テキストを前処理
        text = preprocess_text(text)
        
        # テキストを音声に変換
        speed = float(speed_slider.value)
        audio_path = text_to_speech_hf(text, model_files, speed=speed)
        
        # 音声を表示
        audio_output.clear_output()
        with audio_output:
            display(Audio(audio_path))
        
        # ダウンロードリンクを作成
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        
        download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')
        
        status_output.value = "処理完了！"
    
    except Exception as e:
        status_output.value = f"エラーが発生しました: {str(e)}"

# ダウンロードリンクを作成する関数
def create_download_link(audio_data, filename, text):
    """
    音声データをダウンロードするためのHTMLリンクを作成
    
    Parameters:
    -----------
    audio_data : bytes
        ダウンロードするオーディオデータ
    filename : str
        ダウンロード時のファイル名
    text : str
        リンクに表示するテキスト
        
    Returns:
    --------
    str
        HTMLリンク
    """
    b64 = io.BytesIO(audio_data)
    payload = b64.getvalue()
    import base64
    b64_str = base64.b64encode(payload).decode()
    return f'<a href="data:audio/wav;base64,{b64_str}" download="{filename}">{text}</a>'

# 速度変更時の処理
def on_speed_change(change):
    """
    速度スライダーが変更されたときの処理
    """
    speed_value.value = f"発話速度: {change.new}x"

# UIの作成
upload_button = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='テキストファイルを選択'
)

speed_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='速度:',
    continuous_update=False
)

speed_value = widgets.HTML(value="発話速度: 1.0x")
status_output = widgets.HTML(value="ファイルをアップロードしてください。")
text_preview = widgets.Textarea(
    description='テキスト:',
    placeholder='アップロードされたテキストがここに表示されます',
    disabled=True,
    layout=widgets.Layout(width='100%', height='200px')
)

audio_output = widgets.Output()
download_link = widgets.HTML()

# 注意書きを追加
note_html = widgets.HTML(
    value="<div style='background-color: #fff3cd; padding: 15px; border-radius: 5px; margin: 10px 0;'>"
    "<strong>注意:</strong> このデモではHugging Faceからモデルファイルをダウンロードしますが、"
    "MeloTTSの内部実装へのアクセスが制限されているため、完全な機能を実現するには元のGitHubベースの"
    "アプリケーションを使用することをお勧めします。</div>"
)

# イベントハンドラの登録
upload_button.observe(on_upload_change, names='value')
speed_slider.observe(on_speed_change, names='value')

# UIの表示
display(widgets.HTML("<h1>MeloTTS-Japanese テキスト→音声変換アプリ (Hugging Face版)</h1>"))
display(note_html)
display(widgets.HTML("<p>日本語のテキストファイルをアップロードして、自然な音声に変換します。</p>"))
display(widgets.HBox([upload_button, widgets.VBox([speed_slider, speed_value])]))
display(status_output)
display(text_preview)
display(widgets.HTML("<h3>生成された音声:</h3>"))
display(audio_output)
display(download_link)